Script to download data from openmeteo

In [6]:
import openmeteo_requests

import requests_cache
import pandas as pd
from retry_requests import retry


In [7]:
# Setup the Open-Meteo API client with cache and retry on error
cache_session = requests_cache.CachedSession('.cache', expire_after = -1)
retry_session = retry(cache_session, retries = 5, backoff_factor = 0.2)
openmeteo = openmeteo_requests.Client(session = retry_session)

In [8]:
# Make sure all required weather variables are listed here
# The order of variables in hourly or daily is important to assign them correctly below
url = "https://archive-api.open-meteo.com/v1/archive"
params = {
	"latitude": 41.3,
	"longitude": 2.1,
	"start_date": "2020-01-01",
	"end_date": "2024-05-31",
	"hourly": ["temperature_2m", "apparent_temperature", "relative_humidity_2m", "precipitation", "sunshine_duration", "wind_speed_10m"],
	"timezone": "Europe/Berlin"
}
responses = openmeteo.weather_api(url, params=params)

# Process first location. Add a for-loop for multiple locations or weather models
response = responses[0]
print(f"Coordinates {response.Latitude()}°N {response.Longitude()}°E")
print(f"Elevation {response.Elevation()} m asl")
print(f"Timezone {response.Timezone()}{response.TimezoneAbbreviation()}")
print(f"Timezone difference to GMT+0 {response.UtcOffsetSeconds()} s")

							# Process hourly data. The order of variables needs to be the same as requested.
hourly = response.Hourly()
hourly_temperature_2m = hourly.Variables(0).ValuesAsNumpy()
hourly_apparent_temperature = hourly.Variables(1).ValuesAsNumpy()
hourly_relative_humidity_2m = hourly.Variables(2).ValuesAsNumpy()
hourly_precipitation = hourly.Variables(3).ValuesAsNumpy()
hourly_sunshine_duration = hourly.Variables(4).ValuesAsNumpy()
hourly_wind_speed_10m = hourly.Variables(5).ValuesAsNumpy()

hourly_data = {"date": pd.date_range(
	start = pd.to_datetime(hourly.Time(), unit = "s", utc = True),
	end = pd.to_datetime(hourly.TimeEnd(), unit = "s", utc = True),
	freq = pd.Timedelta(seconds = hourly.Interval()),
	inclusive = "left"
)}

hourly_data["temperature_2m"] = hourly_temperature_2m
hourly_data["apparent_temperature"] = hourly_apparent_temperature
hourly_data["relative_humidity_2m"] = hourly_relative_humidity_2m
hourly_data["precipitation"] = hourly_precipitation
hourly_data["sunshine_duration"] = hourly_sunshine_duration
hourly_data["wind_speed_10m"] = hourly_wind_speed_10m

hourly_dataframe = pd.DataFrame(data = hourly_data)

Coordinates 41.3005256652832°N 2.0659971237182617°E
Elevation 2.0 m asl
Timezone b'Europe/Berlin'b'GMT+1'
Timezone difference to GMT+0 3600 s


In [9]:
# Export the hourly dataframe to csv
hourly_dataframe.to_csv("hourly_data_openmeteo.csv", index = False)
hourly_dataframe

,date,temperature_2m,apparent_temperature,relative_humidity_2m,precipitation,sunshine_duration,wind_speed_10m
0,2019-12-31 23:00:00+00:00,6.527000,3.948897,83.762268,0.0,0.000000,7.895416
1,2020-01-01 00:00:00+00:00,6.477000,3.287031,82.587128,0.0,0.000000,11.753876
2,2020-01-01 01:00:00+00:00,5.477000,2.687171,88.183113,0.0,0.000000,8.942214
3,2020-01-01 02:00:00+00:00,5.277000,2.532773,88.788582,0.0,0.000000,8.496305
4,2020-01-01 03:00:00+00:00,4.877000,2.059424,89.699493,0.0,0.000000,8.669949
...,...,...,...,...,...,...,...
38707,2024-05-31 18:00:00+00:00,19.277000,18.510427,73.282974,0.0,3600.000000,15.111424
38708,2024-05-31 19:00:00+00:00,18.827002,18.943148,77.339714,0.0,2551.825684,10.080000
38709,2024-05-31 20:00:00+00:00,18.227001,19.283100,82.393898,0.0,0.000000,4.680000
38710,2024-05-31 21:00:00+00:00,18.127001,19.108482,82.912766,0.1,0.000000,5.191994
